In [19]:
import os

In [20]:
template = """#!/bin/bash

########## SBATCH Lines for Resource Request ##########

#SBATCH --nodes=1                   # number of different nodes - could be an exact number or a range of nodes (same as -N)
#SBATCH --mem-per-cpu=8G            # memory required per allocated CPU (or core) - amount of memory (in bytes)
#SBATCH --job-name concept_gen_eva  # you can give your job a name for easier identification (same as -J)
#SBATCH --time=3:59:00              # limit of wall clock time - how long the job will run (same as -t)
#SBATCH --cpus-per-task=2           # number of CPUs (or cores) per task (same as -c)
#SBATCH --ntasks=1                  # number of tasks - how many tasks (nodes) that you require (same as -n)
#SBATCH --gres=gpu:v100s:1
#SBATCH -o /mnt/scratch/chenkan4/concept_gen_eva/logfile/sld/%j.log
#SBATCH -e /mnt/scratch/chenkan4/concept_gen_eva/logfile/sld/%j.err

########## Command Lines for Job Running ##########

module load Conda/3
conda activate sld

ROOT_PATH="/mnt/scratch/chenkan4/concept_gen_eva/src/methods/safe-latent-diffusion"
cd ${ROOT_PATH}

JOB_INFO="SLD_GEN"

DATASET="TOKEN_DATASET"
NUM_IMGS=TOKEN_NUM_IMGS
FROM_CASE=TOKEN_FROM_CASE
NUM_PROMPTS=TOKEN_NUM_PROMPTS

COMMAND="python infer.py --dataset $DATASET --num_imgs $NUM_IMGS --from_case $FROM_CASE --num_prompts $NUM_PROMPTS"

#print the information of a job into one file
date >>${ROOT_PATH}/submit_history.log
echo $SLURM_JOB_ID >>${ROOT_PATH}/submit_history.log
echo $JOB_INFO >>${ROOT_PATH}/submit_history.log
echo $COMMAND >>${ROOT_PATH}/submit_history.log

echo "---------------------------------------------------------------" >>${ROOT_PATH}/submit_history.log

echo $JOB_INFO

echo $COMMAND
$COMMAND 1>${ROOT_PATH}/logfile/${SLURM_JOB_ID}_1.log 2>${ROOT_PATH}/logfile/${SLURM_JOB_ID}.err &

wait

echo -e "JobID:$SLURM_JOB_ID 
 JOB_INFO: ${JOB_INFO} 
 Python_command: 
 ${COMMAND}" | mail -s "[Done] ${SLURM_JOB_ID}" harrychen.kr@foxmail.com

date >>${ROOT_PATH}/finish_history.log
echo $SLURM_JOB_ID >>${ROOT_PATH}/finish_history.log
echo $JOB_INFO >>${ROOT_PATH}/finish_history.log
echo $COMMAND >>${ROOT_PATH}/finish_history.log

echo -e "---------------------------------------------------------------" >>${ROOT_PATH}/finish_history.log

scontrol show job $SLURM_JOB_ID     ### write job information to SLURM output file.

"""

In [21]:
dataset = "NSFW"
num_prompts = 200
num_imgs = 5

for from_case in range(0, 2000, 200):
    script_path = f'scripts/{dataset}/{from_case}.sb'
    sbatch = template.replace('TOKEN_DATASET', dataset) \
                .replace('TOKEN_NUM_IMGS', str(num_imgs)) \
                .replace('TOKEN_FROM_CASE', str(from_case)) \
                .replace('TOKEN_NUM_PROMPTS', str(num_prompts))
    
    with open(script_path, 'w') as file:
        file.write(sbatch)
    
    os.system(f'sbatch {script_path}')

Submitted batch job 35134955
Submitted batch job 35134956
Submitted batch job 35134957
Submitted batch job 35134958
Submitted batch job 35134959
Submitted batch job 35134961
Submitted batch job 35134962
Submitted batch job 35134963
Submitted batch job 35134964
Submitted batch job 35134967
